In [172]:
import datetime
import pytz


%run ./common.ipynb

In [173]:
# searchQry='?jql=key%20in%20(%27CSP-151%27,%27CSP-963%27)&expand=changelog'
searchQry='?jql=sprint%20=%207431%20and%20type%20=%20Story&expand=changelog'
searchQry_DS = getDataSet(searchQry)

Getting report results...
https://jira.footlocker.com/rest/api/2/search?jql=sprint%20=%207431%20and%20type%20=%20Story&expand=changelog
<Response [200]>
Report results received...
HTTP 200 - OK


In [174]:
cycleTimeDf = pd.DataFrame(columns=['key','leadTime','development','QA','deployment'])


In [175]:
changelog_Rec =[ ]
for searchResult in searchQry_DS:
    changelog = searchResult['changelog']
    for history in changelog['histories']:
        history_id = history['id']
        created_at = history.get('created',None)
        for item in history['items']:
            # print(item_list['field'])
            changelog_Rec.append({
                'key': searchResult['key'],
                'created_at': created_at,
                'field': item['field'],
                'from': item['fromString'],
                'to': item['toString'],
            })

changelog_Rec_df = pd.DataFrame(changelog_Rec)

status_change = changelog_Rec_df[changelog_Rec_df['field']=='status']

status_change = status_change.reset_index(drop=True)


In [176]:
def calculateCycleTime(_status_change,current_status,key):
    cycleTime = {}
    if (current_status != 'Done'):
        # Get the current time in UTC
        now_utc = datetime.datetime.now(pytz.utc)
        # Convert to Eastern Time
        eastern = pytz.timezone('US/Eastern')
        now_est = now_utc.astimezone(eastern)
        new_node = {
                    'key':key,
                    'created_at': now_est.strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
                    'field': 'status',
                    'from': current_status,
                    'to' : 'Active'
                    }
        _status_change.loc[len(_status_change)] = new_node     

    _status_change = _status_change.reset_index(drop=True)
    _status_change['created_at'] = pd.to_datetime(_status_change['created_at'], format='%Y-%m-%dT%H:%M:%S.%f%z')
    _status_change['time_diff'] = _status_change['created_at'].diff()
    _status_change = _status_change.reset_index(drop=True)
    cycleTime['key'] = _status_change['key'][0]
    cycleTime['leadTime'] = _status_change['time_diff'].sum()
    cycleTime['development'] = _status_change[_status_change['to'].isin( ['In Progress','Code Review','Testing','Active'])]['time_diff'].sum()
    cycleTime['QA'] = _status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum()
    cycleTime['deployment'] = _status_change[_status_change['from'].isin( ['Deploy']) ]['time_diff'].sum()

    display (_status_change)
    display(cycleTime)
    return cycleTime


In [177]:
for searchResult in searchQry_DS:
    result = status_change[status_change['key'] == searchResult['key']]
    cycleTimeDf = cycleTimeDf.append(calculateCycleTime(result,searchResult['fields']['status']['name'],searchResult['key']), ignore_index=True)

display(cycleTimeDf)


{'key': 'CSP-968',
 'created_at': '2024-11-05T18:11:24.284993-0500',
 'field': 'status',
 'from': 'Code Review',
 'to': 'Active'}

/var/folders/sj/ff_l0c9d0072n5r9c61zvf0w0000gq/T/ipykernel_97485/715610537.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _status_change.loc[len(_status_change)] = new_node


,key,created_at,field,from,to
0,CSP-968,2024-10-31T08:24:39.973-0500,status,Open,In Progress
1,CSP-968,2024-11-01T08:47:51.480-0500,status,In Progress,Code Review
2,CSP-968,2024-11-05T18:11:24.284993-0500,status,Code Review,Active


/var/folders/sj/ff_l0c9d0072n5r9c61zvf0w0000gq/T/ipykernel_97485/715610537.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cycleTime['QA'] = _status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum()


,key,created_at,field,from,to,time_diff
0,CSP-968,2024-10-31 08:24:39.973000-05:00,status,Open,In Progress,NaT
1,CSP-968,2024-11-01 08:47:51.480000-05:00,status,In Progress,Code Review,1 days 00:23:11.507000
2,CSP-968,2024-11-05 18:11:24.284993-05:00,status,Code Review,Active,4 days 09:23:32.804993


...

/var/folders/sj/ff_l0c9d0072n5r9c61zvf0w0000gq/T/ipykernel_97485/662442121.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycleTimeDf = cycleTimeDf.append(calculateCycleTime(result,searchResult['fields']['status']['name'],searchResult['key']), ignore_index=True)
/var/folders/sj/ff_l0c9d0072n5r9c61zvf0w0000gq/T/ipykernel_97485/715610537.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cycleTime['QA'] = _status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum()


,key,created_at,field,from,to,time_diff
0,CSP-151,2024-09-18 09:55:23.553000-05:00,status,Open,To Do,NaT
1,CSP-151,2024-09-20 08:47:32.290000-05:00,status,To Do,In Progress,1 days 22:52:08.737000
2,CSP-151,2024-09-23 08:26:42.587000-05:00,status,In Progress,Code Review,2 days 23:39:10.297000
3,CSP-151,2024-09-25 07:54:15.850000-05:00,status,Code Review,Testing,1 days 23:27:33.263000
4,CSP-151,2024-10-23 12:30:57.403000-05:00,status,Testing,Deploy,28 days 04:36:41.553000
5,CSP-151,2024-10-31 13:01:15.170000-05:00,status,Deploy,Done,8 days 00:30:17.767000


{'key': 'CSP-151',
 'leadTime': Timedelta('43 days 03:05:51.617000'),
 'development': Timedelta('6 days 21:58:52.297000'),
 'QA': Timedelta('28 days 04:36:41.553000'),
 'deployment': Timedelta('8 days 00:30:17.767000')}

/var/folders/sj/ff_l0c9d0072n5r9c61zvf0w0000gq/T/ipykernel_97485/662442121.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycleTimeDf = cycleTimeDf.append(calculateCycleTime(result,searchResult['fields']['status']['name'],searchResult['key']), ignore_index=True)


,key,leadTime,development,QA,deployment
0,CSP-968,5 days 09:46:44.311993,5 days 09:46:44.311993,0 days 00:00:00,0 days 00:00:00
1,CSP-963,5 days 21:50:02.087000,1 days 21:16:43.793000,0 days 01:45:34.910000,3 days 20:49:07.297000
2,CSP-961,8 days 04:54:50.781915,7 days 09:58:42.144915,0 days 04:51:44.707915,0 days 00:00:00
3,CSP-960,7 days 09:59:24.330389,7 days 09:59:24.330389,7 days 08:06:18.536389,0 days 00:00:00
4,CSP-956,11 days 20:07:43.510000,6 days 20:35:10.974000,1 days 03:37:20.120000,3 days 19:55:12.416000
5,CSP-950,14 days 06:09:27.658031,11 days 23:37:15.552031,2 days 06:32:12.106000,11 days 01:14:14.458031
6,CSP-945,12 days 09:23:12.586535,11 days 04:44:23.343535,1 days 04:38:49.243000,6 days 04:18:22.393535
7,CSP-938,19 days 04:33:31.775112,11 days 23:24:38.238112,7 days 05:08:53.537000,11 days 01:08:07.168112
8,CSP-930,20 days 00:14:11.150000,2 days 03:05:26.790000,12 days 22:18:04.847000,4 days 22:50:39.513000
9,CSP-929,15 days 03:16:45.036010,15 days 03:16:45.036010,13 days 07:29:47.870010,0 days 00:00:00
